# **Empirische Wirtschaftsforschung**

# Anwendungsbeispiel Flugpreise

**Author:** [Anthony Strittmatter](http://www.anthonystrittmatter.com)

Ihnen steht ein Datensatz mit 3,447 Preisen für unterschiedliche Flüge aus den Jahren 1997, 1998 und 1999 zur Verfügung. 

**Variablenbeschreibung:**
- *lfare*: log(Durchschnittspreis, one-way)
- *mktshr*: Marktanteils des grössten Anbieters auf der Flugstrecke
- *dist*: Distanz, in Meilen
- *passen*: Durchschnittliche Anzahl der Passagiere pro Tag
- *y97*: Dummy für 1997
- *y99*: Dummy für 1999


## Laden der Daten

In [1]:
############## Laden der Daten ##############

data <- read.csv("airfares.csv",header=TRUE, sep=",")

#lfare <- c("lfare")
#restringiert <- c("mktshr", "dist", "passen")
#unrestringiert <- c("mktshr", "dist", "passen", "y97", "y99")

print('Data is loaded.')

##############################################

[1] "Data is loaded."


# Deskriptive Statistiken

Zunächst überprüfen Sie ob der Datensatz vollständig ist und ob es irgendwelche Auffälligkeiten in den Daten gibt.

In [3]:
############## Deskriptive Statistiken ##############

library(psych) # load package
describe(data)

######################################################

,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
lfare,1,3447,5.0765730,0.4410853,5.105946,5.0943686,0.4598357,3.610918,6.198479,2.587561,-0.3079329,-0.439501661,0.007512802
dist,2,3447,989.7449956,611.8536992,861.000000,919.5900689,558.9402000,95.000000,2724.000000,2629.000000,0.8944585,-0.004255814,10.421421184
passen,3,3447,625.4615608,799.6759001,350.000000,454.9612178,250.5594000,4.000000,7637.000000,7633.000000,4.0591270,22.585403373,13.620509897
mktshr,4,3447,0.6129625,0.1961852,0.608600,0.6111361,0.2142357,0.160500,1.000000,0.839500,0.0647639,-0.813012406,0.003341532
y97,5,3447,0.3333333,0.4714729,0.000000,0.2917724,0.0000000,0.000000,1.000000,1.000000,0.7067991,-1.500870196,0.008030380
y99,6,3447,0.3333333,0.4714729,0.000000,0.2917724,0.0000000,0.000000,1.000000,1.000000,0.7067991,-1.500870196,0.008030380


# Konvergenzrate

Sie nehmen eine 50% Unterstichprobe der Daten. Sie untersuchen den Einfluss des MArktanteils auf die Flugpreise. Dazu schätzen sie das einfache Regressionsmodell
\begin{equation*}
lfare = \beta_0 + \beta_1 mktshr + v
\end{equation*}
in der Unterstichprobe und in der geasamten Stichprobe. Überprüfen Sie die die Konvergenzrate der Standardfehler wenn die Stichprobengrösse verdoppelt wird.

In [5]:
############## Konvergenzrate ##############

library(reshape2)

# 50% Unterstichprobe
subsample <- modelr::resample_partition(data, c(A = 0.5, B = 0.5))
subsample <- as.data.frame(subsample$A)

# Einfaches OLS Modell in der Unterstichprobe
sub <- lm(lfare ~ mktshr  , subsample)
summary(sub)

# Einfaches OLS Modell in der gesamten Stichprobe
full <- lm(lfare ~ mktshr  , data)
summary(full)

# Konvergenzrate
print(paste0("Theoretische Konvergenzrate: ", 
             round(sqrt(nrow(subsample)/nrow(data)), digits = 4)))
print(paste0("Tatsächliche Konvergenzrate: ", 
             round(summary(full)$coefficients[2,2]/summary(sub)$coefficients[2,2], digits = 4)))

##############################################


Call:
lm(formula = lfare ~ mktshr, data = subsample)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.20407 -0.28465 -0.00114  0.33924  1.17337 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  5.38754    0.03480   154.8   <2e-16 ***
mktshr      -0.50373    0.05359    -9.4   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.435 on 1721 degrees of freedom
Multiple R-squared:  0.04884,	Adjusted R-squared:  0.04828 
F-statistic: 88.36 on 1 and 1721 DF,  p-value: < 2.2e-16



Call:
lm(formula = lfare ~ mktshr, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.2749 -0.2822  0.0045  0.3340  1.1744 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  5.37866    0.02405  223.61   <2e-16 ***
mktshr      -0.49283    0.03737  -13.19   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.4304 on 3445 degrees of freedom
Multiple R-squared:  0.04805,	Adjusted R-squared:  0.04777 
F-statistic: 173.9 on 1 and 3445 DF,  p-value: < 2.2e-16


[1] "Theoretische Konvergenzrate: 0.707"
[1] "Tatsächliche Konvergenzrate: 0.6974"


# Lagrange Multiplier Test

Als nächstes verwenden Sie ein multivariates Regressionsmodell. Sie sind sich allerdings unschlüssig ob Sie das restringierte Modell,
\begin{equation*}
lfare = \ beta_0 + \beta_1 \cdot mktshr + \beta_2 \cdot dist + \beta_3 \cdot passen + u,
\end{equation*}
oder das unrestingierte Modell,
\begin{equation*}
lfare = \beta_0 + \beta_1 \cdot mktshr + \beta_2 \cdot dist + \beta_3 \cdot passen + \beta_4 \cdot y97+ \beta_5 \cdot y99 + u,
\end{equation*}
zur Schätzung des Flugpreises verwenden sollen. Sie entscheiden sich die Nullhypothese
\begin{equation*}
H_0: \beta_4 =0,  \beta_5 =0,
\end{equation*}
mit Hilfe eines LM-Test zu überprüfen.

## Unrestringiertes Modell

In [48]:
############## Unrestringiertes Modell ##############

unrest_model <- lm(lfare ~ mktshr + dist + passen + y97 + y99 , data)
summary(unrest_model)

#####################################################


Call:
lm(formula = lfare ~ mktshr + dist + passen + y97 + y99, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.33078 -0.24580 -0.02086  0.24305  0.98132 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.453e+00  3.222e-02 138.202   <2e-16 ***
mktshr       2.959e-01  3.520e-02   8.406   <2e-16 ***
dist         4.869e-04  1.130e-05  43.075   <2e-16 ***
passen      -6.162e-05  7.351e-06  -8.384   <2e-16 ***
y97         -2.271e-02  1.415e-02  -1.606    0.108    
y99          1.814e-02  1.416e-02   1.282    0.200    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.339 on 3441 degrees of freedom
Multiple R-squared:  0.4103,	Adjusted R-squared:  0.4094 
F-statistic: 478.8 on 5 and 3441 DF,  p-value: < 2.2e-16


## Restringiertes Modell

In [49]:
############## Restringiertes Modell ##############

rest_model <- lm(lfare ~ mktshr + dist + passen , data)
summary(rest_model)

u_hat <- rest_model$residuals

###################################################


Call:
lm(formula = lfare ~ mktshr + dist + passen, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.35132 -0.24433 -0.01896  0.24259  0.95611 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.452e+00  3.093e-02 143.962   <2e-16 ***
mktshr       2.946e-01  3.520e-02   8.370   <2e-16 ***
dist         4.868e-04  1.131e-05  43.032   <2e-16 ***
passen      -6.106e-05  7.355e-06  -8.303   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.3393 on 3443 degrees of freedom
Multiple R-squared:  0.4089,	Adjusted R-squared:  0.4083 
F-statistic: 793.8 on 3 and 3443 DF,  p-value: < 2.2e-16


## Hilfsregression

In [50]:
############## Hilfregression ##############

auxiliary <- lm(u_hat ~ mktshr + dist + passen + y97 + y99 , data)
summary(auxiliary)

############################################


Call:
lm(formula = u_hat ~ mktshr + dist + passen + y97 + y99, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.33078 -0.24580 -0.02086  0.24305  0.98132 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)
(Intercept)  9.797e-04  3.222e-02   0.030    0.976
mktshr       1.239e-03  3.520e-02   0.035    0.972
dist         1.364e-07  1.130e-05   0.012    0.990
passen      -5.614e-07  7.351e-06  -0.076    0.939
y97         -2.271e-02  1.415e-02  -1.606    0.108
y99          1.814e-02  1.416e-02   1.282    0.200

Residual standard error: 0.339 on 3441 degrees of freedom
Multiple R-squared:  0.002427,	Adjusted R-squared:  0.0009778 
F-statistic: 1.675 on 5 and 3441 DF,  p-value: 0.1372


## Teststatistik

In [52]:
############## LM Teststatistik ##############

print(paste0("R-quadrat: ", round(summary(auxiliary)$r.squared, digits = 4)))

LM = nrow(data)*summary(auxiliary)$r.squared
print(paste0("LM Teststatistik: ", round(LM, digits = 4)))

##############################################

[1] "R-quadrat: 0.0024"
[1] "LM Teststatistik: 8.3672"


## Kritischer Wert aus der Chi-Quadrat Verteilung

In [56]:
############## Kritischer Wert ##############

# 95% Quantil der Chi-Quadrat Verteilung
print(paste0("Kritischer Wert: ", round(qchisq(.95, df=2), digits = 4))) 

# P-Wert (Wahrscheinlichkeit der Nullhypothese)
print(paste0("P-Wert: ", round(1-pchisq(LM, df=2), digits = 5)))


##############################################

[1] "Kritischer Wert: 5.9915"
[1] "P-Wert: 0.01524"


# F-test

Zum Vergleich führen Sie einen zusätzlichen F-test durch um die gleiche Nullhypothese zu überprüfen.

In [43]:
############## F-test ##############

anova(unrest_model, rest_model)

####################################

Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
3441,395.3657,NA,NA,NA,NA
3443,396.3278,-2,-0.9620412,4.186483,0.01527714
